# Convolutional Neural Network & Classification: 

The objective is to build an image classifier that is capable of properly identifying four different categories of image. 

The data consists of various train and test samples across the four categories of image. You will notice that the data for a specific category is a singular image that has been flipped, rotated, or slightly altered in some way. 

You can use the keras package to solve this problem (https://keras.io/)

## 1. Data Processing: 

The train & test data is pretty clean in terms of image data, but we will need to do a bit of prep work to use in our model. 

a) Use the `ImageDataGenerator()` class from `keras.preprocessing.image` to build out an instance called `train_datagen` with the following parameters: 

* rescale = 1./255

* shear_range = 0.2

* zoom_range = 0.2

* horizontal_flip = True

b) Then build your training set by using the method `.flow_from_directory()`

* path (where training data is stored)
* target_size = (64, 64)
* batch_size = 32
* class_mode = categorical 

In [1]:
from keras.preprocessing import image as ki
import numpy as np
import pandas as pd
from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Conv2D, Flatten
from keras.layers import LSTM, Dense, Dropout, MaxPooling2D
from keras.layers.embeddings import Embedding
from keras import backend as k
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
train_datagen = ki.ImageDataGenerator(rescale=1/255, shear_range=.2, 
                                      zoom_range=.2, horizontal_flip=True)

In [3]:
train_datagen = train_datagen.flow_from_directory("./dataset_train/", target_size=(64, 64),
                                 batch_size=32, class_mode="categorical")

Found 88 images belonging to 4 classes.


c) Take a look at your training set: 

What is the image shape of each training observation? Very strange shapes; some crosses, a skull, or letters maybe.

How many total classes do we need to predict on? 4 classes, according to the output of `train_datagen.flow_from_directory`.

2. Initial Classifier Build: Now use `keras` to build an initial image classifier with the following specifications.

Note: If you get lost, there is great documentation online and homework 7 included details on many of the layers used here.

Create an instance of `Sequential` called `classifier`

In [4]:
k.clear_session()
model = Sequential()

* Add a `Conv2D` layer with the following parameters: 

    * filters = 32

    * kernel_size = (3,3)

    * input_shape = image shape found in part 1

    * activation = relu

In [5]:
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                input_shape=(64, 64, 3), activation="relu"))

* Add a `MaxPooling2D` layer where pool_size = (2,2)
* Add a Flatten layer
* Add a Dense layer
    * units = 128
    * activation = relu 

In [6]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))

* Add a final Dense layer (this will output our probabilities):
    * units = # of classes
    
    * activation = softmax 
    
* Compile with the following: 

    * optimize = adam
    
    * loss = categorical cross entropy
    
    * metric = accuracy

In [20]:
model.add(Dense(units=4, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3936384   
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 20        
Total params: 3,937,816
Trainable params: 3,937,816
Non-trainable params: 0
_________________________________________________________________


3. __Model Runs__: This will run various times with different numbers of steps_per_epoch and epochs. 

a) Use `.fit_generator()` with the training set. For the first run, use the following parameters: 

* steps_per_epoch = 10

* epochs = 10

In [8]:
model_fit = model.fit_generator(train_datagen, steps_per_epoch=10, epochs=10)

Epoch 1/10
10/10 [==============================] - 3s 301ms/step - loss: 2.5429 - acc: 0.4918
Epoch 2/10
10/10 [==============================] - 3s 286ms/step - loss: 0.4509 - acc: 0.8803
Epoch 3/10
10/10 [==============================] - 3s 282ms/step - loss: 0.2062 - acc: 0.9688
Epoch 4/10
10/10 [==============================] - 3s 295ms/step - loss: 0.1233 - acc: 0.9865
Epoch 5/10
10/10 [==============================] - 3s 300ms/step - loss: 0.0778 - acc: 0.9824
Epoch 6/10
10/10 [==============================] - 3s 291ms/step - loss: 0.0648 - acc: 0.9814
Epoch 7/10
10/10 [==============================] - 3s 299ms/step - loss: 0.0388 - acc: 0.9896
Epoch 8/10
10/10 [==============================] - 3s 287ms/step - loss: 0.0221 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 3s 282ms/step - loss: 0.0200 - acc: 0.9968
Epoch 10/10
10/10 [==============================] - 3s 301ms/step - loss: 0.0101 - acc: 1.0000


b) Write out each model & `model_weights` to a file. An example is below:

```python
# write model and model weights to disk
model_yaml = classifier.to_yaml()
with open("model_1.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
    classifier.save_weights("model_1.h5")
    print("Saved model to disk")
```

In [9]:
# write model and model weights to disk
model_yaml = model.to_yaml()
with open("model_1.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
    model.save_weights("model_1.h5")
    print("Saved model to disk")

Saved model to disk


c) Predict using the model built in step 2. An example below shows how to reread weights & model: 

In [31]:
import os, glob
import numpy as np
from keras.models import model_from_yaml
from keras.preprocessing import image

# Load model from disk
yaml_file = open("model_4.yaml", "r")
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)

# Load weights into new model
model.load_weights("model_4.h5")
print("Loaded model from disk")

# Test data path
img_dir = "./dataset_test" # Enter Directory of all images

# Iterate over each test image
# Make a prediction and add to results 
data_path = os.path.join(img_dir, "*g")
files = glob.glob(data_path)
data = []
results = []

for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

results

Loaded model from disk


[array([3]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([1]),
 array([1])]

d) Determine accuracy.

Note: To determine accuracy, you will need to manually check the labels given to each class in the training data. This will require you to go and look in the training data, and then determine how a category was coded in `keras`.



In [32]:
actual_label_list = [0, 0, 2, 2, 1, 1, 3, 3]
predicted = [a for r in results for a in r]
actual_label_list == predicted
accuracy = len([i for i, j in zip(actual_label_list, predicted) if i == j]) / len(actual_label_list)
print(f"Model accuracy: {accuracy * 100}%\n")

Model accuracy: 25.0%



After you do this, you will need to compare predicted values to the actual values for the test set (there are only 8 test observations). 

e) Run this process for the following combinations:

* (steps_per_epoch: 10, epochs: 10) <- the one we just did 

* (steps_per_epoch: 10, epochs: 20)

* (steps_per_epoch: 10, epochs: 30)

* (steps_per_epoch: 30, epochs: 10)

* (steps_per_epoch: 30, epochs: 20)

* (steps_per_epoch: 30, epochs: 30)

* (steps_per_epoch: 50, epochs: 10)

* (steps_per_epoch: 50, epochs: 20)

* (steps_per_epoch: 50, epochs: 30)

* (steps_per_epoch: 50, epochs: 100) (Please note: This one will take some time so you should consider running and saving the model outputs so you don't have to keep an eye on your code)

In [16]:
# http://danshiebler.com/2016-09-14-parallel-progress-bar/
def parallel_process(array, function, n_jobs=8, use_kwargs=False, front_num=3):
    """
        A parallel version of the map function with a progress bar. 

        Args:
            array (array-like): An array to iterate over.
            function (function): A python function to apply to the elements of array
            n_jobs (int, default=16): The number of cores to use
            use_kwargs (boolean, default=False): Whether to consider the elements of array as dictionaries of 
                keyword arguments to function 
            front_num (int, default=3): The number of iterations to run serially before kicking off the parallel job. 
                Useful for catching bugs
        Returns:
            [function(array[0]), function(array[1]), ...]
    """
    # We run the first few iterations serially to catch bugs
    if front_num > 0:
        front = [function(**a) if use_kwargs else function(a) for a in array[:front_num]]
    # If we set n_jobs to 1, just run a list comprehension. This is useful for benchmarking and debugging.
    if n_jobs==1:
        return front + [function(**a) if use_kwargs else function(a) for a in tqdm(array[front_num:])]
    # Assemble the workers
    with ProcessPoolExecutor(max_workers=n_jobs) as pool:
        # Pass the elements of array into function
        if use_kwargs:
            futures = [pool.submit(function, **a) for a in array[front_num:]]
        else:
            futures = [pool.submit(function, a) for a in array[front_num:]]
        kwargs = {
            'total': len(futures),
            'unit': 'it',
            'unit_scale': True,
            'leave': True
        }
        # Print out the progress as tasks complete
        for f in tqdm(as_completed(futures), **kwargs):
            pass
    out = []
    # Get the results from the futures. 
    for i, future in tqdm(enumerate(futures)):
        try:
            out.append(future.result())
        except Exception as e:
            out.append(e)
    return front + out

In [14]:
def fit_model(params):
    print(f"Fitting model: {params}")
    model_fit = model.fit_generator(train_datagen, steps_per_epoch=params["steps_per_epoch"], epochs=params["epochs"])
    # write model and model weights to disk
    model_yaml = model.to_yaml()
    model_name = params["model_name"]
    with open(f"{model_name}.yaml", "w") as yaml_file:
        yaml_file.write(model_yaml)
        model.save_weights(f"{model_name}.h5")
        print("Saved model to disk")

def predict_image(model_name):
    # Load model from disk
    yaml_file = open(f"{model_name}.yaml", "r")
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    model = model_from_yaml(loaded_model_yaml)

    # Load weights into new model
    model.load_weights(f"{model_name}.h5")
    print("Loaded model from disk")

    # Test data path
    img_dir = "./dataset_test" # Enter Directory of all images

    # Iterate over each test image
    # Make a prediction and add to results 
    data_path = os.path.join(img_dir, "*g")
    files = glob.glob(data_path)
    data = []
    results = []

    for f1 in files:
        img = image.load_img(f1, target_size = (64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        data.append(img)
        result = model.predict(img)
        r = np.argmax(result, axis=1)
        results.append(r)

    return(results)

def model_accuracy(result):
    actual_label_list = [0, 0, 2, 2, 1, 1, 3, 3]
    predicted = [a for r in results for a in r]
    accuracy = len([i for i, j in zip(actual_label_list, predicted) if i == j]) / len(actual_label_list)
    return(accuracy)

def run_model(params):
    model_name = params["model_name"]
    fit_model(params)
    result = predict_image(model_name)
    return(model_accuracy(result))

In [17]:
param_grid = [{"steps_per_epoch": 10, "epochs": 10, "model_name": "model_1"},
 {"steps_per_epoch": 10, "epochs": 20, "model_name": "model_2"},
 {"steps_per_epoch": 10, "epochs": 30, "model_name": "model_3"},
 {"steps_per_epoch": 30, "epochs": 10, "model_name": "model_4"},
 {"steps_per_epoch": 30, "epochs": 20, "model_name": "model_5"},
 {"steps_per_epoch": 30, "epochs": 30, "model_name": "model_6"},
 {"steps_per_epoch": 50, "epochs": 10, "model_name": "model_7"},
 {"steps_per_epoch": 50, "epochs": 20, "model_name": "model_8"},
 {"steps_per_epoch": 50, "epochs": 30, "model_name": "model_9"},
 {"steps_per_epoch": 50, "epochs": 100, "model_name": "model_10"}]

In [21]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3936384   
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 20        
Total params: 3,937,816
Trainable params: 3,937,816
Non-trainable params: 0
_________________________________________________________________


In [25]:
model_grid_results = parallel_process(param_grid, run_model, n_jobs=1)

Fitting model: {'steps_per_epoch': 10, 'epochs': 10, 'model_name': 'model_1'}
Epoch 1/10
10/10 [==============================] - 3s 318ms/step - loss: 1.1457 - acc: 0.4242
Epoch 2/10
10/10 [==============================] - 3s 288ms/step - loss: 1.0840 - acc: 0.4956
Epoch 3/10
10/10 [==============================] - 3s 305ms/step - loss: 1.0713 - acc: 0.5041
Epoch 4/10
10/10 [==============================] - 3s 288ms/step - loss: 1.0616 - acc: 0.4997
Epoch 5/10
10/10 [==============================] - 3s 326ms/step - loss: 1.0479 - acc: 0.5028
Epoch 6/10
10/10 [==============================] - 3s 296ms/step - loss: 1.0340 - acc: 0.4858
Epoch 7/10
10/10 [==============================] - 3s 302ms/step - loss: 1.0180 - acc: 0.5044
Epoch 8/10
10/10 [==============================] - 3s 299ms/step - loss: 1.0080 - acc: 0.4928
Epoch 9/10
10/10 [==============================] - 3s 322ms/step - loss: 0.9998 - acc: 0.5098
Epoch 10/10
10/10 [==============================] - 3s 285ms/step 


  0%|          | 0/7 [00:00<?, ?it/s]

Loaded model from disk
Fitting model: {'steps_per_epoch': 30, 'epochs': 10, 'model_name': 'model_4'}
Epoch 1/10
30/30 [==============================] - 9s 291ms/step - loss: 0.5188 - acc: 1.0000
Epoch 2/10
30/30 [==============================] - 8s 274ms/step - loss: 0.4977 - acc: 1.0000
Epoch 3/10
30/30 [==============================] - 9s 297ms/step - loss: 0.4773 - acc: 1.0000
Epoch 4/10
30/30 [==============================] - 8s 259ms/step - loss: 0.4581 - acc: 1.0000
Epoch 5/10
30/30 [==============================] - 9s 286ms/step - loss: 0.4410 - acc: 1.0000
Epoch 6/10
30/30 [==============================] - 8s 275ms/step - loss: 0.4236 - acc: 1.0000
Epoch 7/10
30/30 [==============================] - 9s 300ms/step - loss: 0.4089 - acc: 1.0000
Epoch 8/10
30/30 [==============================] - 9s 299ms/step - loss: 0.3926 - acc: 1.0000
Epoch 9/10
30/30 [==============================] - 9s 292ms/step - loss: 0.3777 - acc: 1.0000
Epoch 10/10
30/30 [=========================


 14%|█▍        | 1/7 [01:26<08:39, 86.59s/it]

Loaded model from disk
Fitting model: {'steps_per_epoch': 30, 'epochs': 20, 'model_name': 'model_5'}
Epoch 1/20
30/30 [==============================] - 8s 279ms/step - loss: 0.3520 - acc: 1.0000
Epoch 2/20
30/30 [==============================] - 9s 299ms/step - loss: 0.3394 - acc: 1.0000
Epoch 3/20
30/30 [==============================] - 9s 295ms/step - loss: 0.3279 - acc: 1.0000
Epoch 4/20
30/30 [==============================] - 9s 295ms/step - loss: 0.3168 - acc: 1.0000
Epoch 5/20
30/30 [==============================] - 9s 308ms/step - loss: 0.3066 - acc: 1.0000
Epoch 6/20
30/30 [==============================] - 9s 310ms/step - loss: 0.2961 - acc: 1.0000
Epoch 7/20
30/30 [==============================] - 11s 369ms/step - loss: 0.2866 - acc: 1.0000
Epoch 8/20
30/30 [==============================] - 9s 302ms/step - loss: 0.2770 - acc: 1.0000
Epoch 9/20
30/30 [==============================] - 9s 300ms/step - loss: 0.2681 - acc: 1.0000
Epoch 10/20
30/30 [========================

...


 86%|████████▌ | 6/7 [22:56<04:23, 263.91s/it]

Loaded model from disk
Fitting model: {'steps_per_epoch': 50, 'epochs': 100, 'model_name': 'model_10'}
Epoch 1/100
50/50 [==============================] - 13s 263ms/step - loss: 0.0144 - acc: 1.0000
Epoch 2/100
50/50 [==============================] - 14s 283ms/step - loss: 0.0140 - acc: 1.0000
Epoch 3/100
50/50 [==============================] - 14s 287ms/step - loss: 0.0136 - acc: 1.0000
Epoch 4/100
50/50 [==============================] - 13s 260ms/step - loss: 0.0133 - acc: 1.0000
Epoch 5/100
50/50 [==============================] - 14s 275ms/step - loss: 0.0129 - acc: 1.0000
Epoch 6/100
50/50 [==============================] - 14s 278ms/step - loss: 0.0126 - acc: 1.0000
Epoch 7/100
50/50 [==============================] - 14s 290ms/step - loss: 0.0123 - acc: 1.0000
Epoch 8/100
50/50 [==============================] - 15s 292ms/step - loss: 0.0120 - acc: 1.0000
Epoch 9/100
50/50 [==============================] - 16s 316ms/step - loss: 0.0117 - acc: 1.0000
Epoch 10/100
50/50 [====

50/50 [==============================] - 13s 265ms/step - loss: 0.0019 - acc: 1.0000
Epoch 84/100
50/50 [==============================] - 14s 273ms/step - loss: 0.0019 - acc: 1.0000
Epoch 85/100
50/50 [==============================] - 14s 274ms/step - loss: 0.0018 - acc: 1.0000
Epoch 86/100
50/50 [==============================] - 14s 273ms/step - loss: 0.0018 - acc: 1.0000
Epoch 87/100
50/50 [==============================] - 14s 273ms/step - loss: 0.0017 - acc: 1.0000
Epoch 88/100
50/50 [==============================] - 14s 271ms/step - loss: 0.0017 - acc: 1.0000
Epoch 89/100
50/50 [==============================] - 14s 277ms/step - loss: 0.0016 - acc: 1.0000
Epoch 90/100
50/50 [==============================] - 14s 277ms/step - loss: 0.0016 - acc: 1.0000
Epoch 91/100
50/50 [==============================] - 14s 279ms/step - loss: 0.0016 - acc: 1.0000
Epoch 92/100
50/50 [==============================] - 17s 332ms/step - loss: 0.0015 - acc: 1.0000
Epoch 93/100
50/50 [=============


100%|██████████| 7/7 [47:14<00:00, 622.16s/it]


Loaded model from disk


In [35]:
pd.DataFrame(param_grid).assign(accuracy=model_grid_results)

,epochs,model_name,steps_per_epoch,accuracy
0,10,model_1,10,0.25
1,20,model_2,10,0.25
2,30,model_3,10,0.25
3,10,model_4,30,0.25
4,20,model_5,30,0.25
5,30,model_6,30,0.25
6,10,model_7,50,0.25
7,20,model_8,50,0.25
8,30,model_9,50,0.25
9,100,model_10,50,0.25


## Conceptual Questions: 

4. Discuss the effect of the following on accuracy and loss (train & test): 

    * Increasing the `steps_per_epoch`: 

    * Increasing the number of `epochs`: 

5. Name two uses of zero padding in CNN. 

    * Zero padding preserves information as we add more layers to the CNN.
    
    * Zero padding is also used to set dimensions equal to our output specifications.

6. What is the use of a 1 x 1 kernel in CNN? [Stack Exchange](https://stats.stackexchange.com/questions/194142/what-does-1x1-convolution-mean-in-a-neural-network): We use the 1x1 convolutional filters to reduce dimensionality in the filter dimension. As I explained above, these 1x1 conv layers can be used in general to change the filter space dimensionality (either increase or decrease) and in the Inception architecture we see how effective these 1x1 filters can be for dimensionality reduction, explicitly in the filter dimension space, not the spatial dimension space.

7. What are the advantages of a CNN over a fully connected DNN for this image classification problem? They are good at extracting features and efficiently processing images.